In [1]:
from selenium import webdriver
from PIL import Image
import os
from os import path
import time
import random
import pandas as pd
import requests
import json
import numpy as np

In [2]:
import code.visualizations as viz
import code.data_preparation as dprep

## Load API keys

In [1]:
def get_api_keys():
    '''Returns the keys for calls to 3 apis. The key is stored in secret file on local computer'''
    with open('C:/Users/james/.secret/patronOmdb.txt', 'r') as f:
        omdb_key = f.read()

    with open('C:/Users/james/.secret/imdbApi.txt', 'r') as g:
        imdb_key = g.read()
        
    return (omdb_key, imdb_key, tmdb_key)

In [4]:
api_keys = get_api_keys()
omdb_key = api_keys[0]
imdb_key = api_keys[1]
tmdb_key = api_keys[2]


## Scraping functions

### Scrape financial details from the numbers website and make a list of lists

In [6]:
def the_numbers_scraping():
    '''Srapes the Numbers webite with Selenium, returns a list of lists, each containing financial details for a movie'''
    chromedriver = 'C:/Users/james/AppData/Local/Google/Chrome/chromedriver.exe'
    os.environ["webdriver.chrome.driver"] = chromedriver
    driver = webdriver.Chrome(chromedriver)
    financials_list = []
    #scroll through 62 pages
    for r in range(62):
        time.sleep(random.uniform(2, 4))
        if r == 0:
            url = 'https://www.the-numbers.com/movie/budgets/all'
        else:
            url = 'https://www.the-numbers.com/movie/budgets/all/' + str(r) + '01'
        driver.get(url)  
        mytable = driver.find_element_by_css_selector('table')
        for row in mytable.find_elements_by_css_selector('tr'):
            row_list=[]
            cn=0
            for cell in row.find_elements_by_tag_name('td'):
                if cn != 0:  
                    row_list.append(cell.text)
                cn+=1
            if row_list:
                financials_list.append(row_list)
    return financials_list

In [7]:
financials_list = the_numbers_scraping()
financials_list[:3]

[['Apr 23, 2019',
  'Avengers: Endgame',
  ' $400,000,000',
  ' $858,373,000',
  ' $2,797,800,564'],
 ['May 20, 2011',
  'Pirates of the Caribbean: On Stranger Tides',
  ' $379,000,000',
  ' $241,071,802',
  ' $1,045,713,802'],
 ['Apr 22, 2015',
  'Avengers: Age of Ultron',
  ' $365,000,000',
  ' $459,005,868',
  ' $1,395,316,979']]

In [8]:
with open("data/financials_list.json", "w") as ffp:
        json.dump(financials_list, ffp)

### Make numbers_df, from the list of lists. Filter out corrupt/anomalous rows by rejecting movies with no worldwide takings and movies with budget < 1m dollars and only include movies with presence in the US

In [9]:
def construct_prepare_numbers_df(financials_list):
    '''Takes in the Numbers list of lists, returns a dataframe and formats data'''
    numbers_df = pd.DataFrame(financials_list)
    numbers_df.columns=['release_date', 'title', 'budget_$', 'domestic_box_office_$', 'worldwide_box_office_$']

    # strip '$' ' '  ','  from values that need to be floats
    numbers_df= numbers_df.applymap(lambda x: x.strip())
    numbers_df= numbers_df.applymap(lambda x: x[1:].replace(',', '') if  x[0] == "$" else x)
    
    # create colums to break up the release date
    numbers_df.insert(1, "release_year", "")
    numbers_df.insert(2, "release_month", "")
    numbers_df.insert(3, "release_day", "")
    
    #split up the date into d/m/y
    no_date_list = []
    for index, row in numbers_df.iterrows():
        try:
            date_list=row['release_date'].replace(',', '').split()
            row['release_year'] = date_list[2]
            row['release_month'] = date_list[0]
            row['release_day'] = date_list[1]
        except:
            no_date_list.append([row['title'], date_list])
            
    numbers_df = numbers_df[['title', 'release_year', 'release_month', 'release_day', 'budget_$', 'domestic_box_office_$', 'worldwide_box_office_$']]
    
    # insert dummy date if not present to allow int coversion
    for index, row in numbers_df.iterrows():
        if row['release_year'] == '':
            row['release_year'] = '0'
        
    # make columns numeric
    numbers_df[['release_year', 'budget_$', 'domestic_box_office_$', 'worldwide_box_office_$']] = numbers_df[['release_year', 'budget_$', 'domestic_box_office_$', 'worldwide_box_office_$']].astype('int64')
    
    #filter out any movies with less than $1m budget
    numbers_df = numbers_df[numbers_df['budget_$'] >1000000]
    
    # filter out any films where worldwide figures suggest unusable film, and where domestic 
    # takings suggest it was only a hit outside the US
    
    numbers_df = numbers_df[numbers_df['domestic_box_office_$'] > 0]
    numbers_df = numbers_df[numbers_df['worldwide_box_office_$'] > 0]
        
    return numbers_df

In [10]:
numbers_df = construct_prepare_numbers_df(financials_list)

In [11]:
numbers_df.to_csv('data/numbers_df.csv', index=False)

### Make financials_df, which is a copy of numbers_df but with profit/loss columns added and filtered to include only movies released in last 15 years. Also adds empty column added to hold imdb_code


In [12]:
def make_fin_15(numbers_df):
    '''Takes in the Numbers dataframe and filters to only include movies from last 15 years and add profit, imdb_code columns'''
    financials_15_df = numbers_df.copy()
    
    # make imdb_code column that will be primary key and used to join to the attributes df later
    financials_15_df.insert(0, "imdb_code", "No_code")
       
    # make profit/loss columns - gross and return on investment and column of domestic share of takings
    financials_15_df['profit_loss_$'] = financials_15_df['worldwide_box_office_$'] - financials_15_df['budget_$']
    financials_15_df['return_pct'] = (100 / financials_15_df['budget_$']) * financials_15_df['profit_loss_$']
    financials_15_df['domestic_%'] = 100 * (financials_15_df['domestic_box_office_$'] / financials_15_df['worldwide_box_office_$'])
    
    # filter for only last 15 years
    financials_15_df = financials_15_df[financials_15_df['release_year'] >= 2007]
    
    return financials_15_df

In [13]:
financials_15_df = make_fin_15(numbers_df)

## API functions

### Use omdb api to get attributes for movies in numbers_df. The response is stored in global dictionary omdb_attrs that will be used to create the attributes_df dataframe. It also updates the imdb_code column in financials_15_df. It also makes a list of failed calls, due to ambiguous title

In [14]:
omdb_attrs = {}
dud_url=[]
retrieve_error=[]

def on_success(financials_15_df, movie_data, movie):
    imdb_id = movie_data['imdbID'] 
    financials_15_df['imdb_code'][financials_15_df.title == movie] = imdb_id
    keys = ['Actors', 'Director', 'Genre', 'Plot', 'Rated', 'Ratings', 'Runtime', 'Writer', 'Title', 'imdbRating', 'imdbVotes']
    omdb_attrs[imdb_id] = {x:movie_data[x] for x in keys}

def url_maker(year, title, omdb_key):
    url = 'http://www.omdbapi.com/?t=' + title + '&y=' + year + '&apikey=' + omdb_key
    return url


def get_page(url):
    response=requests.get(url)
    movie_data = json.loads(response.content.decode('utf-8')) 
    return movie_data

def make_omdb_dict(financials_15_df, omdb_key):
    '''The engine of the OMDB API calls. It iterates through each movie in financials_15_df and retrieves its metadata from 
    omdb. It returns a dictionary of dictionaries containing metadata for each film. It also adds the imdb_code to 
    financials_15_df and constructs a list of movies where the API call fails due to an ambiguous title
    The important films from that list are manually stored in the manual_list variable below, and their imdb_codes manually obtained from 
    imdb website. They are then fed into the following functions to perform the same tasks with the ambiguously titled movies'''
    for index, row in financials_15_df.iterrows():
        movie = row['title']
        title = movie.replace(',', '%2C').replace (' ', '+').replace (':', '%3A')
        year = str(row['release_year'])
        url = url_maker(year, title, omdb_key)
        try:
            movie_data = get_page(url)
            if "Error" in movie_data: 
                #try previous release year
                prev_year = (str(int(year) -1))
                url = url_maker(prev_year, title, omdb_key)
                movie_data = get_page(url)  
                if "Error" in movie_data:
                    film_error = movie + " - " + movie_data_t['Error'] + "\n"
                    retrieve_error.append(prev_film_error)
                else:
                    on_success(financials_15_df, movie_data, movie)
            else:
                on_success(financials_15_df, movie_data, movie)  
        except:
            dud_url.append(movie)

    return (omdb_attrs, financials_15_df)


update_dud_url=[]
update_retrieve_error=[]


In [15]:
(omdb_attrs, financials_15_df) = make_omdb_dict(financials_15_df, omdb_key)

<ipython-input-14-1388f0215367>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  financials_15_df['imdb_code'][financials_15_df.title == movie] = imdb_id


In [16]:
omdb_attrs

{'tt4154796': {'Actors': 'Robert Downey Jr., Chris Evans, Mark Ruffalo',
  'Director': 'Anthony Russo, Joe Russo',
  'Genre': 'Action, Adventure, Drama',
  'Plot': "After the devastating events of Avengers: Infinity War (2018), the universe is in ruins. With the help of remaining allies, the Avengers assemble once more in order to reverse Thanos' actions and restore balance to the universe.",
  'Rated': 'PG-13',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.4/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '94%'},
   {'Source': 'Metacritic', 'Value': '78/100'}],
  'Runtime': '181 min',
  'Writer': 'Christopher Markus, Stephen McFeely, Stan Lee',
  'Title': 'Avengers: Endgame',
  'imdbRating': '8.4',
  'imdbVotes': '937,294'},
 'tt1298650': {'Actors': 'Johnny Depp, Penélope Cruz, Ian McShane',
  'Director': 'Rob Marshall',
  'Genre': 'Action, Adventure, Fantasy',
  'Plot': 'Jack Sparrow and Barbossa embark on a quest to find the elusive fountain of youth, only to disco

In [17]:
financials_15_df.head(2)

,imdb_code,title,release_year,release_month,release_day,budget_$,domestic_box_office_$,worldwide_box_office_$,profit_loss_$,return_pct,domestic_%
0,tt4154796,Avengers: Endgame,2019,Apr,23,400000000,858373000,2797800564,2397800564,599.450141,30.680278
1,tt1298650,Pirates of the Caribbean: On Stranger Tides,2011,May,20,379000000,241071802,1045713802,666713802,175.913932,23.053325


### The list of movies with failed calls is filtered to only include important movies -- stored in manual_list. Their imdb codes are found manually and stored in manual_imdb. They are zipped together and used to once again call the omdb api  and update the attributes dictionary (you can call omdb api either using title or imdb code) .  <br> The API call is therefore made differently, and the source variables that make up the URL come from a different data structure, so easier to use new functions rather than adapt the main function

In [18]:
manual_list = ['Star Wars Ep. VII: The Force Awakens', 
'Star Wars Ep. VIII: The Last Jedi', 
'Prince of Persia: Sands of Time', 
'Fast and Furious 6', 
'The Chronicles of Narnia: The Voyage of the Daw…', 
'Harry Potter and the Deathly Hallows: Part II', 
'Harry Potter and the Deathly Hallows: Part I', 
'Fantastic Four: Rise of the Silver Surfer', 
'The Hangover 3', 
'Ford v. Ferrari', 
'Dr. Seuss’ The Grinch', 
'Mamma Mia: Here We Go Again!', 
'The Angry Birds Movie', 
'Wall Street 2: Money Never Sleeps',
'Dr. Seuss’ The Lorax', 
'All Eyez on Me', 
'John Wick: Chapter Two', 
'Underworld 3: Rise of the Lycans', 
'Halloween 2',        
'Precious (Based on the Novel Push by Sapphire)']

manual_imdb = ['tt2488496', 'tt2527336', 'tt0473075', 'tt1905041', 'tt0980970', 'tt1201607', 'tt0926084', 'tt0486576', 'tt1951261', 'tt1950186', 'tt2709692', 'tt6911608', 'tt1985949', 'tt1027718', 'tt1482459', 'tt1666185', 'tt4425200', 'tt0834001', 'tt1311067', 'tt0929632']

zipped_codes = list(zip(manual_list, manual_imdb))

In [19]:
def update_on_success(financials_15_df, movie_data, pair, imdb_code):
    title = pair[0]
    financials_15_df.loc[financials_15_df['title'] == title, 'imdb_code'] = imdb_code
    keys = ['Actors', 'Director', 'Genre', 'Plot', 'Rated', 'Ratings', 'Runtime', 'Writer', 'Title', 'imdbRating', 'imdbVotes']
    omdb_attrs[imdb_code] = {x:movie_data[x] for x in keys}

def update_get_page(url):
    response=requests.get(url)
    movie_data = json.loads(response.content.decode('utf-8')) 
    return movie_data

def update_make_omdb_dict(financials_15_df, omdb_key):
    '''Similar to preceding main api call function, but called afterwards for those movies that failed due to ambiguous title
    using imdb code instead of title. Easier to use totally new functions due to the different way the url is constructed'''
    for pair in zipped_codes:
        imdb_code = pair[1]
        url = 'http://www.omdbapi.com/?i=' + imdb_code + '&apikey=' + omdb_key
        try:
            movie_data = update_get_page(url)
            update_on_success(financials_15_df, movie_data, pair, imdb_code) 
        except:
            update_dud_url.append(pair[0])

     # delete any rows in financials_15yr where there is no imdb_code

    financials_15_df = financials_15_df[financials_15_df.imdb_code != 'No_code']
    return financials_15_df      

In [20]:
financials_15_df = update_make_omdb_dict(financials_15_df, omdb_key)

### Turn the attributes dict into a dataframe

In [21]:
def rating_formatter():
    '''updates attributes dict returned in above functions by exploding the ratings list, so we have a column for each site'''
    for key in omdb_attrs:
        for k in omdb_attrs[key]['Ratings']:
            outlet_ratings = list(k.values())
            if outlet_ratings[0] == 'Rotten Tomatoes':
                omdb_attrs[key]['RottenRating'] = outlet_ratings[1][:-1]
            elif outlet_ratings[0] == 'Metacritic':
                omdb_attrs[key]['MetacriticRating'] = outlet_ratings[1][:-4]


def clean_values():
    '''formats the dictionary for conversion to df '''
    for mov in omdb_attrs:
        actor_list = omdb_attrs[mov]['Actors'].split(',')
        actor_list = [ac.strip() for ac in actor_list]
        omdb_attrs[mov]['Actor_List'] = actor_list
        
        genre_list = omdb_attrs[mov]['Genre'].split(',')
        genre_list = [ge.strip() for ge in genre_list]
        omdb_attrs[mov]['Genre_List'] = genre_list
        
        writer_list = omdb_attrs[mov]['Writer'].split(',')
        writer_list = [wr.strip() for wr in writer_list]
        omdb_attrs[mov]['Writer_List'] = writer_list
        
        director_list = omdb_attrs[mov]['Director'].split(',')
        director_list=[di.strip() for di in director_list]
        omdb_attrs[mov]['Director_List'] = director_list
        
        try:
            runtime = int(omdb_attrs[mov]['Runtime'].split(' ')[0])
            omdb_attrs[mov]['Run_Time'] = runtime
        
        except:
            omdb_attrs[mov]['Run_Time'] = 0
        
        try:
            imdb_votes = int(omdb_attrs[mov]['imdbVotes'].replace(',',''))
            omdb_attrs[mov]['imdb_votes'] = imdb_votes
        except:
            omdb_attrs[mov]['imdb_votes'] = '0'


def make_attributes_df():
    '''makes the dataframe of movie metadata from ommdb'''
    rating_formatter()
    clean_values()
    
    # make attributes linking dict 
    to_df_dict = {}
    att_keys = ['Genre_List', 'Actor_List', 'Director_List', 'Writer_List',
          'Rated', 'Run_Time','MetacriticRating', 'RottenRating', 'imdbRating',
          'imdb_votes']
    for m in omdb_attrs:
        to_df_dict[m] = {}
        
        for ks in att_keys:
            try:
                to_df_dict[m][ks] = omdb_attrs[m][ks]
            except:
                to_df_dict[m][ks] = np.nan 

     
    #make the actual attributes_df  
    attributes_df = pd.DataFrame(to_df_dict).transpose()
    attributes_df.reset_index(inplace=True)
    attributes_df = attributes_df.rename(columns = {'index':'imdb_code'})

    # make numeric values and make a mean ratings column
    attributes_df['imdbRating'] = attributes_df['imdbRating'].replace('N/A',np.NaN)
    attributes_df[['RottenRating', 'MetacriticRating', 'imdbRating', 'imdb_votes', 'Run_Time']] = attributes_df[['RottenRating', 'MetacriticRating', 'imdbRating', 'imdb_votes', 'Run_Time']].astype('float')

    attributes_df['imdbRating'] = attributes_df['imdbRating'].map(lambda x: 10*x)

    attributes_df['mean_rating'] =  (attributes_df['imdbRating'] +  attributes_df['RottenRating'] +  attributes_df['MetacriticRating'])/3
    
    return attributes_df

In [22]:
attributes_df = make_attributes_df()

## Save the dataframes, attributes dictionary and scraped list

In [23]:
def save_files(financials_list, numbers_df, financials_15_df, omdb_attrs, attributes_df):
    '''Saves raw dataframes at the end of session in which they were created'''
    numbers_df.to_csv('data/numbers_df.csv', index=False)
    financials_15_df.to_csv('data/financials_15_df.csv', index=False)
    attributes_df.to_csv('data/attributes_df.csv', index=False)
    
    with open("data/financials_list.json", "w") as ffp:
        json.dump(financials_list, ffp)

    with open("data/omdb_attrs.json", "w") as ffo:
        json.dump(omdb_attrs, ffo)

In [25]:
save_files(financials_list, numbers_df, financials_15_df, omdb_attrs, attributes_df)

**DATAFRAMES** <br><br>
**numbers_df** - all financial data from The Numbers website<br>
**financials_15_df** - all financial data for numbers movies in last 15 years <br>
**financials_hits_df** - same data but filtered out loss making movies, those only succesful outside english speaking world and those with no documented figures <br>
**financials_flops_df** - same but filtered out profit making movies <br>
<br>